# 光学式文字認識(OCR)

<p style='text-align:center'><img src='./images/ocr.jpg' alt='新聞を読むロボット'/></p>

一般的なコンピュータビジョンの課題は、画像中のテキストを検出して解釈することです。この種の処理は、しばしば*光学式文字認識* (OCR) と呼ばれています。

## コンピュータビジョンサービスを使って画像中のテキストを読み取る

**Computer Vision**コグニティブサービスは、以下のようなOCRタスクをサポートします。

- 複数の言語のテキストを読み取るために使用できる **OCR** API。この API は同期的に使用でき、画像内の少量のテキストを検出して読み取る必要がある場合に有効です。
- 大きな文書に最適化された **Read** API。このAPIは非同期で使用され、印刷されたテキストと手書きのテキストの両方に使用することができます。

このサービスは、**Computer Vision**リソースまたは**Cognitive Services**リソースを作成することで使用できます。

まだ作成していない場合は、Azure サブスクリプションで **Cognitive Services** リソースを作成してください。

1. 別のブラウザタブで、https://portal.azure.com の Azure ポータルを開き、Microsoft アカウントでサインインします。
2. **[&#65291;リソースの作成]**ボタンをクリックして、*Cognitive Services* を検索し、次の設定で**Cognitive Services**リソースを作成します。:
    - **Name**: *一意の名前を入力してください*.
    - **Subscription**: *Azureサブスクリプション*.
    - **Location**: *利用可能なリージョンを選択します*.
    - **Pricing tier**: S0
    - **Resource group**: *一意な名前を持つリソースグループを作成します*.
3. デプロイが完了するのを待ちます。次に、Cognitive Servicesリソースに移動し、**クイックスタート** ページで、キーとエンドポイントに注意してください。クライアント アプリケーションからCognitive Services リソースに接続するには、これらが必要です。

### Cognitive Servicesのキーとエンドポイント用の変数を作成します。

Cognitive Servicesリソースに接続するためには、キーとエンドポイントが必要です。

1. Azureポータルで、Cognitive Servicesリソースの**クイックスタート**ページを表示します。
2. リソースの**Key1**をコピーして、**YOUR_COG_KEY**を置き換えて、以下のコードに貼り付けます。
3. リソースの **endpoint** をコピーして、**YOUR_COG_ENDPOINT**.を置き換えて、以下のコードに貼り付けます。
4. 下のセルの緑色の<span style="color:green">&#9655;</span>ボタンをクリックして、コードを実行します。


In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

これでCognitive Servicesのセットアップが完了しました。

そしてPythonから本演習を実行するには、まずAzureの関連パッケージをインストールする必要があります。

In [ ]:
! pip install azure-cognitiveservices-vision-computervision

これで、コンピュータビジョンサービスを使用して画像内のテキストを読み取る準備が整いました。

まず、**OCR** API を使用して、画像を同期的に解析し、画像に含まれるテキストを読み取ることができます。この例では、Northwind Tradersという架空の小売会社の広告画像があり、その中にテキストが含まれています。それを読み取るために以下のセルを実行してください。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# computer visionサービスのクライアントを獲得する
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# 画像ファイルを読み込む
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Computer Visionサービスを利用して，画像中のテキストを検索します．
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# テキストを一行ずつ処理する
for region in read_results.regions:
    for line in region.lines:

        # テキストの行の中の単語を読む
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# 画像を開いて表示します。
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

画像内で見つかったテキストは、領域、行、単語の階層構造に整理されており、コードはこれらを読み込んで結果を取得します。

結果では、画像の上に読み込んだテキストを表示します。

## バウンディングボックスを表示する

結果には、画像内で見つかったテキストの行と個々の単語の*バウンディングボックス*座標も含まれています。以下のセルを実行して、上記で取得した広告画像のテキスト行のバウンディングボックスを確認してください。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# 画像ファイルを読み込む
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# コンピュータビジョンサービスのクライアントを獲得する
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# 画像内の印刷された文字を読み込んで操作IDを取得するリクエストを送信する
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# 非同期操作が完了するのを待ちます
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# 操作が成功した場合は、テキストを1行ずつ処理します。
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# 画像を開いて表示します。
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

結果として、テキストの各イネのバウンディングボックスが画像上に矩形で表示されます。

## Read APIを使用する

以前に使用した OCR API は、少量のテキストを含む画像でも問題なく動作します。スキャンしたドキュメントなど、より大きなテキストを読み取る必要がある場合は、**Read** APIを使用できます。このためには、複数のステップのプロセスが必要です。

1. コンピュータビジョンサービスに画像を送信して、非同期で読み取りと分析を行います。
2. 解析操作が完了するのを待ちます。
3. 解析結果を取得します。

次のセルを実行して、このプロセスを使用して、Northwind Tradersの店長にスキャンした手紙のテキストを読み取る。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import TextOperationStatusCodes, TextRecognitionMode
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
im

結果を確認してください。手書きの署名と印刷されたテキストのほとんどが構成されている手紙の完全な転写があります。手紙の元の画像は、OCRの結果の下に表示されています（見るにはスクロールする必要があるかもしれません）。

## 手書きのテキストを読む

先の例では、画像を解析するリクエストで、*印刷された*テキストに対して動作を最適化するテキスト認識モードが指定されていました。にもかかわらず、手書きの署名が読み取られたことに注意してください。しかし、主に*手書き*の文書を分析する必要がある場合は、テキスト認識モードを手書きに最適化するように設定することができます。

たとえば、買い物リストを含むノートを書き、携帯電話のアプリを使用してノートを読み取り、その中に含まれるテキストを書き写すとします。

以下のセルを実行して、手書きの買い物リストの読み取り操作の例を見てみましょう。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# 画像ファイルを読み込む
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# コンピュータビジョンサービスのクライアントを獲得する
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# 画像内の手書き文字を読みたいというリクエストを出して、操作IDを取得する
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# 非同期操作が完了するのを待ちます
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# 操作が成功した場合は、テキストを1行ずつ処理します。
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# 画像を開いて表示します。
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## More Information

OCR のためのコンピュータビジョンサービスの使用については、[コンピュータビジョンのドキュメント]（https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/concept-recognizing-text）を参照してください。